# Everyday-Use Version of the RAG-pipeline
* Please run this code to start the pipeline.
* After that copy and paste the path of the document you would like to correct in the next cell. 

## Try this version without saving any files

In [15]:
from docx import Document
from llama_index.core import StorageContext, load_index_from_storage, Settings
from llama_index.llms.openai import OpenAI
import openai
from dotenv import load_dotenv
import os

def correct_document(file_path):
    # Setup
    load_dotenv()
    openai.api_key = os.getenv('OPENAI_API_KEY')
    Settings.llm = OpenAI(model="gpt-4", temperature=0.3)
    
    # Read document
    doc = Document(file_path)
    text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
    
    # Load index and create query engine
    index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./RAG_index"))
    
    # Process correction
    correction_prompt = f"""
    Correct the mistakes of the provided text. The text was written by a student. The student should have a language level A2-B1 according
    to Common European Framework of Reference for Languages: Learning, Teaching, Assessment CEFRL. Correct the mistakes and provide the adequate solutions.
    Don't change the original input text. Add only the version with the corrections to your answer. Only use this structure to correct the text:

    Example 1: "Yo te llama José."
    Corrected example 1: "Yo te llama José. (Yo me llamo José.)"
    Example 2: "Las coches es caros."
    Corrected example 2: "Las coches es caros. (Los coches son caros.)"
    Example 3: "Hola María, yo te llama José."
    Corrected example 3: "Hola María, yo te llama José. (Hola María, me llamo José.) Me gusta los caballo. (Me gustan los caballos.)"

    Please correct this text and provide one sentence objective and simple with feedback and which grammar topics the student should improve.
    The feedback and explanations have to be in german.
    Feedback structure:
    "Feedback: [insert feedback sentence here]
    An diesem Grammatikthema/diesen Grammatikthemen solltest du noch arbeiten: [insert grammar topics here]
    Beurteilungsraster A2:
    - Erfüllung der Aufgabenstellung (EA): [insert chosen descriptions]
    - Aufbau und Layout (AL): [insert chosen descriptions]
    - Spektrum Sprachlicher Mittel (SSM): [insert chosen descriptions]
    - Sprachrichtigkeit (SR): [insert chosen descriptions]"

    Text to correct: {text}
    """
    
    return index.as_query_engine().query(correction_prompt)

# Usage
print(correct_document('./data/correo electronico.docx'))

Correo electronico 

De: adrian123@gmail.com
Para: padres123@gmail.com
Fecha: 12 de novembre de 2024 (12 de noviembre de 2024)
Asunto: me ciudad favorita (mi ciudad favorita)

Queridos padres,

Estoy en mi ciuadad favorita, Barcelona. (Estoy en mi ciudad favorita, Barcelona.) La ciudad es muy bonita y hay mucha gente viviendo aquí. Además, hace mucho calor aquí en Barcelona.

Hoy he visto la ciudad. Para el almuerzo probe tapas en un pequeno restaurante local. (Para el almuerzo probé tapas en un pequeño restaurante local.) ! Fue delicioso! (¡Fue delicioso!) Me gusta de la playa y comer. (Me gusta la playa y comer.) Luego pasee por el parque y pasee por la playa de la noche. (Luego paseé por el parque y por la playa en la noche.) Mas tarde me relaje en la playa de la Barcelona. (Más tarde me relajé en la playa de Barcelona.)

Manana tengo planes emocionantes. (Mañana tengo planes emocionantes.) Quiero visitar muchos mercados y restaurantes para probar mas comida tipica. (Quiero visitar 

## Correction without saving
* **Copy and paste the path** of the document you would like to correct in the **cell below**. 

In [20]:
#response = correct_document('./data/correo electronico.docx')
response = correct_document('./data/SP blog comment.docx')
print(response)

Escrito por: Sofia Y, el 23 de febrero
¡Hola Maria y todos los seguidoros del blog! (¡Hola Maria y todos los seguidores del blog!)
Acabo de leer tu entrada de blog y me ha gustado.  Fue muy interesante.  en este momento hay algunas tendencias de moda. (En este momento hay algunas tendencias de moda.) Por ejemplo: las cintas y la ropa holgada. Me gustan las cintas, porque es muy bonita. (Me gustan las cintas, porque son muy bonitas.) Mejora cada atuendo. Me gusta la ropa holgada, porque me siento cómodo. Odio los jeans ajustados, porque es feo. (Odio los jeans ajustados, porque son feos.)

Las redes sociales afectan mucho a la moda. Por ejemplo: TikTok e Instagram. Muchos influencers influyen a los espectadores cuando usan una prenda hermosa y quieren comprarla de nuevo. (Muchos influencers influyen en los espectadores cuando usan una prenda hermosa y quieren comprarla de nuevo.) Esto crea moda rápida. 

En mi vida la moda rápida y la mode sostenible es muy importante. (En mi vida, la m

## Correction of text + word-document 
* Please run this code to start the pipeline
* After that copy and paste the path of the document you would like to correct
* It will be saved to `.corrected_texts` -folder
### Install the package if not already available

In [46]:
!pip install python-docx

In [28]:
from docx import Document
from llama_index.core import StorageContext, load_index_from_storage, Settings
from llama_index.llms.openai import OpenAI
import openai
from dotenv import load_dotenv
import os
from datetime import datetime
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import pathlib

def correct_document(file_path):
    # Setup
    load_dotenv()
    openai.api_key = os.getenv('OPENAI_API_KEY')

    # Define system prompt
    system_prompt = """You are a Spanish language teacher with expertise in CEFR levels A2-B1. 
    Your role is to:
    1. Identify and correct Spanish language errors
    2. Provide constructive feedback in German
    3. Evaluate texts according to CEFR standards
    4. Maintain a professional and educational tone
    5. Focus on common grammatical structures at A2-B1 level"""

    # Initialize OpenAI with system prompt
    Settings.llm = OpenAI( model="gpt-4", temperature=0.3,
    system_prompt=system_prompt)
        
    # Create output directory if it doesn't exist
    output_dir = pathlib.Path('./corrected_texts')
    output_dir.mkdir(exist_ok=True)
    
    # Read document
    doc = Document(file_path)
    text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
    
    # Load index and create query engine
    index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./RAG_index"))
    
    # Process correction
    correction_prompt = f"""
    Correct the mistakes of the provided text. The text was written by a student. The student should have a language level A2-B1 according
    to Common European Framework of Reference for Languages: Learning, Teaching, Assessment CEFRL. Correct the mistakes and provide the adequate solutions.
    Don't change the original input text. Only use this structure to correct the text:

    Example 1: "Yo te llama José."
    Corrected example 1: "Yo te llama José. (Yo me llamo José.)"
    Example 2: "Las coches es caros."
    Corrected example 2: "Las coches es caros. (Los coches son caros.)"
    Example 3: "Hola María, yo te llama José."
    Corrected example 3: "Hola María, yo te llama José. (Hola María, me llamo José.) Me gusta los caballo. (Me gustan los caballos.)"

    Please correct this text and provide one sentence objective and simple with feedback and which grammar topics the student should improve.
    The feedback and explanations have to be in german.
    
    Feedback structure:
    "Feedback*: [insert feedback sentence here]
    
    An diesem Grammatikthema/diesen Grammatikthemen solltest du noch arbeiten: [insert grammar topics here]
    \n
    Diesem Niveau würde der Text entstprechen [use your knowledge to chose a level according to Common European Framework of Reference for Languages: Learning, Teaching, Assessment CEFRL]
    \n
    Kriterien des Beurteilungsrasters:
    - Erfüllung der Aufgabenstellung (EA): [insert chosen descriptions]
    - Aufbau und Layout (AL): [insert chosen descriptions]
    - Spektrum Sprachlicher Mittel (SSM): [insert chosen descriptions]
    - Sprachrichtigkeit (SR): [insert chosen descriptions]"

    Text to correct: {text}
    """
    
    response = index.as_query_engine().query(correction_prompt)
    
    # Generate timestamp for unique filenames
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    original_filename = pathlib.Path(file_path).stem
    
    # Create Word document
    doc_output = Document()
    
    # Add title
    title = doc_output.add_heading('Texto corregido', 0)
    title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    # Add correction and feedback
    doc_output.add_heading('Corrección y feedback:', level=1)
    doc_output.add_paragraph(str(response))
    
    # Save as Word document
    docx_path = output_dir / f'{original_filename}_corrected_{timestamp}.docx'
    doc_output.save(docx_path)
    
    print(f"Correction saved to:\n- {docx_path}\n")
    return response

# Usage
if __name__ == "__main__":
    response = correct_document('./data/correo electronico.docx')
    print("\nCorrection Result:")
    print(response)


Correction saved to:
- corrected_texts\correo electronico_corrected_20250130_200943.docx


Correction Result:
"Correo electronico 

De: adrian123@gmail.com
Para: padres123@gmail.com
Fecha: 12 de novembre de 2024 (Fecha: 12 de noviembre de 2024)
Asunto: me ciudad favorita (Asunto: mi ciudad favorita)

Queridos padres,

Estoy en mi ciuadad favorita, Barcelona. (Estoy en mi ciudad favorita, Barcelona.) La ciudad es muy bonita y hay mucha gente viviendo aquí. Además, hace mucho calor aquí en Barcelona.

Hoy he visto la ciudad. Para el almuerzo probe tapas en un pequeno restaurante local. (Para el almuerzo probé tapas en un pequeño restaurante local.) ! Fue delicioso! (¡Fue delicioso!) Me gusta de la playa y comer. (Me gusta la playa y comer.) Luego pasee por el parque y pasee por la playa de la noche. (Luego paseé por el parque y por la playa en la noche.) Mas tarde me relaje en la playa de la Barcelona. (Más tarde me relajé en la playa de Barcelona.)

Manana tengo planes emocionantes. (Ma

## SUPER FAST VERSION
* First you have to run the RAG-pipeline cell once.
* Change the path to correct a different text.
* It will be saved to `.corrected_texts` -folder

In [30]:
response = correct_document('./data/correo electronico.docx')

Correction saved to:
- corrected_texts\correo electronico_corrected_20250126_141934.docx



In [35]:
response = correct_document('./data/SP blog comment.docx')

Correction saved to:
- corrected_texts\SP blog comment_corrected_20250126_142548.docx



## Creating exercises

In [ ]:
#Add exercises to docx-file --> mistakes of the student


In [30]:
from docx import Document
from llama_index.core import StorageContext, load_index_from_storage, Settings
from llama_index.llms.openai import OpenAI
import openai
from dotenv import load_dotenv
import os

def correct_document(file_path):
    # Setup
    load_dotenv()
    openai.api_key = os.getenv('OPENAI_API_KEY')
    
    # Define system prompt
    system_prompt = """You are an expert language teacher specialized in Spanish with expertise in CEFR levels A2-B1. 
    Your task is to correct Spanish texts and provide creative and useful tasks for language learners. 
    """
    
    # Initialize OpenAI with system prompt
    Settings.llm = OpenAI(
        model="gpt-4", 
        temperature=0.3,
        system_prompt=system_prompt
    )
    
    # Read document
    doc = Document(file_path)
    text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
    
    # Load index and create query engine
    index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./RAG_index"))
    
    # Process correction
    correction_prompt = f"""
    You will receive a Spanish text written by a student and must:
    Identify all grammar, spelling, and usage errors in the text.
    Create two concise, targeted grammar exercises based on the errors found. 
    These exercises should help the writer practice the specific grammatical points that were problematic in the original text.
    
    Be sure to: 
    1.) Keep your explanations brief but clear. 
    2.) Present the exercises in a way that they can be used for independent practice (e.g., fill-in-the-blanks, sentence rewriting, short multiple-choice, etc.).
    3.) Write your entire response in Spanish, except for any special instructions or formatting. 
    4.) If explanations or special instructions are needed use german language. 

    Structure of your output:
    " 
    EJERCICIOS PERSONALIZADOS:
    Tus errores: [insert the grammar topics in spanish where you detected errors]
    Ejercicio 1: [insert exercise 1]
    Ejercicio 2: [insert exercise 2]

    SOLUCIONES:

    Soluciones del ejercicio 1: [insert only correct answers of exercise 1]
    Soluciones del ejercicio 2: [insert only correct answers of exercise 2]
    "

    Text to correct: {text}
    """
    
    return index.as_query_engine().query(correction_prompt)

# Usage
print(correct_document('./data/correo electronico.docx'))

EJERCICIOS PERSONALIZADOS:
Tus errores: Uso incorrecto de preposiciones, falta de tildes, errores de concordancia y errores de ortografía.

Ejercicio 1: Completa las siguientes oraciones con la preposición correcta.
1. Estoy ____ mi ciudad favorita.
2. Paseé ____ el parque.
3. Me relajé ____ la playa.

Ejercicio 2: Corrige los errores de ortografía y tildes en las siguientes palabras.
1. ciuadad
2. probe
3. pequeno
4. delicioso
5. Souvenirs

SOLUCIONES:

Soluciones del ejercicio 1: 
1. Estoy en mi ciudad favorita.
2. Paseé por el parque.
3. Me relajé en la playa.

Soluciones del ejercicio 2: 
1. ciudad
2. probé
3. pequeño
4. delicioso
5. souvenirs
